In [1]:
from database.market import Market
from database.strategy import Strategy
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None
import pytz

In [2]:
market = Market()
strategy = Strategy()

In [3]:
market.connect()
prices = market.retrieve("prices")
market.disconnect()

In [4]:
strategy.connect()
weekly_sim = strategy.retrieve("weekly_sim")
strategy.disconnect()

In [5]:
prices["date"] = pd.to_datetime(prices["date"])
prices["year"] = [x.year for x in prices["date"]]
prices["quarter"] = [x.quarter for x in prices["date"]]
prices["week"] = [x.week for x in prices["date"]]

In [7]:
seats = 6
strategy.connect()
for number_of_training_weeks in tqdm(range(14,68,14)):
    for weekly_model_training_year in tqdm(range(1,6)):
        for weekly_category_training_year in tqdm(range(1,5)):
            for value in [True,False]:
                current_weekly_sim = weekly_sim[(weekly_sim["mty"]==weekly_model_training_year) & \
                                               (weekly_sim["tw"]==number_of_training_weeks) & \
                                               (weekly_sim["cty"]==weekly_category_training_year)]
                final_sim = prices.merge(current_weekly_sim,on=["year","quarter","week","ticker"],how="left").dropna().reset_index()
                final_sim["weekly_delta"] = (final_sim["weekly_price_regression_prediction"] - final_sim["adjClose"]) / final_sim["adjClose"]
                if not value:
                    final_sim["weekly_delta"] = final_sim["weekly_delta"] * -1
                final_sim.rename(columns={"score":"weekly_score"},inplace=True)
                start = final_sim["date"].min()
                end = final_sim["date"].max()
                date = start
                final_sim.sort_values("date",ascending=True,inplace=True)
                for delta in range(0,15,5):
                    d = float(delta/100)
                    for score in range(70,100,10):
                        for seat in range(seats):
                            date = start
                            while date <= end:
                                req = float(score/100)
                                if date.weekday() > 4 or date >= datetime(date.year,12,17).astimezone(pytz.utc):
                                    date = date + timedelta(days=1)
                                else:
                                    try:
                                        todays_sim = final_sim[(final_sim["date"]==date) 
                                                               & (final_sim["weekly_delta"] > d) \
                                                              & (final_sim["weekly_score"] >= req)].sort_values("weekly_delta",ascending=False)
                                        if todays_sim.index.size < 1 or seat > todays_sim.index.size:
                                            date = date + timedelta(days=1)
                                        else:
                                            try:
                                                trade = todays_sim.iloc[seat]
                                                ticker = trade["ticker"]
                                                buy_price = trade["adjClose"]
                                                exit_price = buy_price * (1+d)
                                                max_days = min(14,(datetime(date.year,12,17).astimezone(pytz.utc)-date).days)
                                                max_date = date + timedelta(days=max_days)
                                                exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                                    (final_sim["date"] > date) & \
                                                                  (final_sim["date"] <= max_date) & \
                                                                  (final_sim["adjClose"] >= exit_price)].sort_values("date")
                                                if exits.index.size < 1:
                                                    exits = final_sim[(final_sim["ticker"]==ticker) & 
                                                                  (final_sim["year"] == date.year) & \
                                                                (final_sim["date"] > max_date)].sort_values("date")
                                                    sell_trade = exits.iloc[0]
                                                else:
                                                    sell_trade = exits.iloc[0]
                                                trade["sell_date"] = sell_trade["date"]
                                                trade["sell_price"] = sell_trade["adjClose"]
                                                trade["delta"] = (trade["sell_price"] - trade["adjClose"]) / trade["adjClose"]
                                                trade["score"] = score
                                                trade["d"] = delta
                                                trade["wcty"] = weekly_category_training_year
                                                trade["wmty"] = weekly_model_training_year
                                                trade["tw"] = number_of_training_weeks
                                                trade["value"] = value
                                                trade["seat"] = seat
                                                strategy.store("exit_fixed_trades",pd.DataFrame([trade]))
                                                date = trade["sell_date"] + timedelta(days=1)
                                            except Exception as e:
                                                date = date + timedelta(days=1)
                                    except Exception as e:
                                        date = date + timedelta(days=1)
strategy.disconnect()

  0%|                                                           | 0/5 [00:00<?, ?it/s]

  0%|                                                           | 0/4 [00:00<?, ?it/s]

 25%|███████████▊                                   | 1/4 [25:56<1:17:48, 1556.15s/it]

 50%|████████████████████████▌                        | 2/4 [51:43<51:41, 1550.86s/it]

 75%|███████████████████████████████████▎           | 3/4 [1:18:02<26:03, 1563.86s/it]

100%|███████████████████████████████████████████████| 4/4 [1:43:35<00:00, 1553.87s/it]

 20%|█████████                                    | 1/5 [1:43:35<6:54:21, 6215.50s/it]

  0%|                                                           | 0/4 [00:00<?, ?it/s]

 25%|███████████▊                                   | 1/4 [25:20<1:16:02, 1520.70s/it]

 50%|████████████████████████▌                        | 2/4 [50:35<50:33, 1516.94s/it]

 75%|███████████████████████████████████▎           | 3/4 [1:15:59<25:20, 1520.61s/it]

100%|███████████████████████████